# Game of Life

The [game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) simulates the dynamics of some living species according to a few simple rules devised by the British mathematician [J. H. Conway](https://en.wikipedia.org/wiki/John_Horton_Conway). The dynamics unfolds on a square lattice of cells which can either be populated or unpopulated. The rules for the time evolution are as follows:

1. A populated cell will become unpopulated in the next generation if it has less than two
   populated neighbors.
1. A populated cell will be populated in the next generation if it has two or three 
   populated neighbors.
1. A populated cell will become unpopulated in the next generation if it has more than    
   three populated neighbors.
1. An unpopulated cell will become populated in the next generation if exactly three of its  
   neighbors are populated.
   
These rules can be motivated by a minimum of populated neighboring cells needed for reproduction and a maximum of populated neighboring cells to avoid starvation from overpopulation. The rules can be stated in a simpler form as:

A cell will be populated in the next generation if

* it is presently populated and has two populated neighboring cells
* independently of its present state, it has three populated neighboring cells

In all other cases, the cell will be unpopulated in the next generation. The update from generation to generation has to be done for all cells in a single step.

In a first step, one needs to implement an update to the next generation in an efficient way. We store the present state of the population on the lattice in a Numpy array containing integers where a populated cell is denoted by 1 while an unpopulated cell is denoted by 0. A second Numpy array of the same size is generated where each entry contains the number of populated neighbors of that entry in the original array.

### Number of populated neighboring cells

In order to determine how many of the 8 neighboring cells are populated, we make use of [`scipy.signal.convolve2d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html#scipy.signal.convolve2d) from the [Scipy library](https://docs.scipy.org/doc/scipy/reference/). Take a look at the documentation to familiarize yourself with the `convolve2d` function. If you are not familiar with the concept of convolution, take a look [here](~Convolution.ipynb).

In the following, in order to avoid boundary effects, we will assume periodic boundary conditions.

In [ ]:
# put imports here if necessary
### BEGIN SOLUTION
import numpy as np
from scipy import signal
### END SOLUTION

def neighbors(world):
    """determine the next generation in the game of life
    
       The state of the population is stored in the 2d array
       'world'. The number of neighbors is obtained by means
       of scipy.signal.convolve2d with periodic boundary
       conditions.
       
    """
    ### BEGIN SOLUTION
    v = np.array([[1, 1, 1],
                  [1, 0, 1],
                  [1, 1, 1]])
    return signal.convolve2d(world, v, mode='same', boundary='wrap')
    ### END SOLUTION

Test your solution by executing the following two cells.

In [ ]:
result = neighbors(np.zeros((10, 10), dtype=np.int8))
assert result is not None, 'It seems that you do not return a result.'
assert isinstance(result, np.ndarray), 'It seems that you do not return a Numpy array.'

In [ ]:
import numpy as np
a = np.random.randint(0, 2, (5, 5))
result = neighbors(a)
assert np.all(0 <= result), 'The number of neighbors cannot be negative.'
assert np.all(result <= 8), 'The number of neighbors cannot be larger than 8.'
for nx, ny in ((1, 1), (0, 1), (1, 0), (0, 0)):
    msg = 'Error in element ({}, {})\n'.format(nx, ny)
    msg = msg+'original matrix:\n{}\n'.format(a)
    msg = msg+'neighbors:\n{}'.format(result)
    assert result[nx, ny] == (a[nx-1, ny-1]+a[nx-1, ny]+a[nx-1, ny+1]
                              +a[nx, ny-1]+a[nx, ny+1]
                              +a[nx+1, ny-1]+a[nx+1, ny]+a[nx+1, ny+1]), msg
old_convolve2d = signal.convolve2d
del signal.convolve2d
try:
    neighbors(a)
except AttributeError:
    pass
else:
    raise AssertionError("It seems that scipy.signal.convolve2d is not used.")
finally:
    signal.convolve2d = old_convolve2d
    del old_convolve2d

### Create the next generation

Knowing how to efficiently determine the number of populated neighboring cells allows us now to update a given configuration stored in the area `world` by means of Conway's rules stated above.

The following hints might be useful:

* A test for equality can be performed on a complete array, resulting in a Boolean array.
* Boolean `and` and `or` can be performed on whole arrays by means of `&` and `|`, 
  respectively.
* Avoid creating new arrays as this is expensive in compute time.

In [ ]:
# put imports here if necessary
### BEGIN SOLUTION
import numpy as np
from scipy import signal
### END SOLUTION

def update(world):
    """determine the next generation in the game of life
    
       The state of the population is stored in the 2d array
       'world'. The number of neighbors is obtained by means
       of scipy.signal.convolve2d with periodic boundary
       conditions.
       
    """
    ### BEGIN SOLUTION
    v = np.array([[1, 1, 1],
                  [1, 0, 1],
                  [1, 1, 1]])
    nr_neighbors = signal.convolve2d(world, v, mode='same', boundary='wrap')
    world = world & (nr_neighbors == 2)
    world = world | (nr_neighbors == 3)
    return world
    ### END SOLUTION

Check your implementation by executing the following two cells. The configurations used in the tests in the second cell can be found [here](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

In [ ]:
result = update(np.zeros((10, 10), dtype=np.int8))
assert result is not None, 'It seems that you do not return a result.'
assert isinstance(result, np.ndarray), 'It seems that you do not return a Numpy array.'

In [ ]:
def msg(text, found, expected):
    return ('The update of the {} configuation is not correct'
            '\nfound:\n{}\nexpected:\n{}\n'.format(text, found, expected))

expected = np.array([[0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0],
                     [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0],
                     [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0]])
result = update(expected)
assert np.all(result == expected), msg('loaf', result, expected)
result = update(np.array([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0],
                          [0, 1, 1, 1, 0], [0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0]]))
expected = np.array([[0, 0, 0, 0, 0], [0, 0, 1, 0, 0],
                     [0, 0, 1, 0, 0], [0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0]])
assert np.all(result == expected), msg('blinker', result, expected)
result = update(np.array([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0],
                          [1, 1, 0, 0, 0], [0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0]]))
expected = np.array([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0],
                     [1, 1, 1, 0, 0], [0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0]])
assert np.all(result == expected), msg('glider', result, expected)

### Graphical representation of the population dynamics

Now let us construct a class called `Conway` which allows us to visualize the time evolution of a given population. The class should have the following methods:

* `__init__`   
  This method should obtain information about the grid size to be used as well as the initial
  configuration which could either be a random pattern or a predefined pattern placed at a 
  specified position.
* `set_initial`
  This method should be called by `__init__` in order to set the initial state.
* `update`
  This method is called by `animate` and should yield the configuration corresponding to the
  next generation.
* `animate`
  This method is provided. Once the class is instantiated, the `animate` method needs to be 
  called to obtain a graphical representation of the population dynamics.
  
Implement the following initial configurations, but feel also free to implement more of them. Search the internet for inspiration.

![Initial configurations](initials.png)

In [ ]:
%matplotlib notebook
# add imports here, if necessary
### BEGIN SOLUTION
import numpy as np
from scipy import signal
### END SOLUTION
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class Conway:
    """Conway's game of life
    
    """
    # put here a dictionary named 'configs' which contains initial configurations
    # blinker, toad, glider, beacon, pulsar, pentadecathlon
    ### BEGIN SOLUTION
    configs = {'blinker': [[0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0],
                           [0, 1, 1, 1, 0],
                           [0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0]],
               'toad': [[0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 1, 1, 0],
                        [0, 1, 1, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0]],
               'glider': [[0, 0, 0, 0, 0, 0],
                          [0, 0, 1, 0, 0, 0],
                          [0, 0, 0, 1, 0, 0],
                          [0, 1, 1, 1, 0, 0],
                          [0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0]],
               'beacon': [[0, 0, 0, 0, 0, 0],
                          [0, 1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 0, 0],
                          [0, 0, 0, 1, 1, 0],
                          [0, 0, 0, 1, 1, 0],
                          [0, 0, 0, 0, 0, 0]],
               'pulsar': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
               'pentadecathlon': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
              }
    ### END SOLUTION
    
    # define here the array required for the convolution
    ### BEGIN SOLUTION
    v = np.array([[1, 1, 1],
                  [1, 0, 1],
                  [1, 1, 1]])
    ### END SOLUTION
    
    def __init__(self, size=100, initial=None, x0=0, y0=0):
        """prepare initial state of game of life
        
           size: Size of the square lattice on which the population
                 dynamics unfolds. Periodic boundary conditions are
                 used.
           initial: Can either be None, in which case a random population
                 is used, or a valid string designating a predefined
                 configuration.
           x0, y0: Position where the upper left corner of a predefined
                configuation should be placed on a lattice which is 
                originally unpopulated.
        
        """
        ### BEGIN SOLUTION
        self.size = size
        self.set_initial(initial, x0, y0)
        ### END SOLUTION
        
    def set_initial(self, initial, x0, y0):
        """set the initial configuation
        
           initial: Can either be None, in which case a random population
                 is used, or a valid string designating a predefined
                 configuration. A KeyError exception is raised if the
                 given string is not valid.
           x0, y0: Position where the upper left corner of a predefined
                configuation should be placed on a lattice which is 
                originally unpopulated.           
        
        """
        ### BEGIN SOLUTION
        if initial is None:
            self.world = np.random.randint(0, 2, (self.size, self.size))
        else:
            try:
                config = self.configs[initial]
            except KeyError:
                raise KeyError('initial configuration {} not found'.format(initial))
            else:
                self.world = np.zeros((self.size, self.size), dtype=np.int8)
                self.world[x0:x0+len(config), y0:y0+len(config[0])] = config
        ### END SOLUTION
                
    def update(self, *args):
        """update the population in self.world
        
        """
        ### BEGIN SOLUTION
        n = signal.convolve2d(self.world, self.v, mode='same', boundary='wrap')
        self.world = self.world & (n == 2)
        self.world = self.world | (n == 3)
        ### END SOLUTION
        self.im.set_array(self.world)
        return self.im,
    
    def animate(self, **kwargs):
        """display an animation of the population dynamics
        
           Keyword arguments given to this method are forwarded to
           matplotlib.animation.FuncAnimation for further evaluation
           
        """
        fig = plt.figure()
        self.im = plt.imshow(self.world, animated=True)
        self.anim = FuncAnimation(fig, self.update, **kwargs)
        plt.show()

#### Random initial state

In [ ]:
c = Conway(200)
c.animate()

### Specified initial state

In [ ]:
c = Conway(100, 'blinker', 20, 20)
c.animate()